In [7]:
import numpy as np

In [8]:
data_ibl = np.genfromtxt('20240204_1426.txt', skip_header=0, usecols=(1,3,4,5))
time_ibl = data_ibl[:, 0]
omegaX_ibl = data_ibl[:, 1]
omegaY_ibl = data_ibl[:, 2]
omegaZ_ibl = data_ibl[:, 3]

In [9]:
data_1 = np.genfromtxt('05022024_1.txt', skip_header=0, max_rows=None)
time_1 = data_1[:, 0]
fi_1 = data_1[:, 1]
theta_1 = data_1[:, 2]
omega_1 = theta_1*3600

In [10]:
import matplotlib.pyplot as plt

In [11]:
from scipy.optimize import minimize
from scipy.interpolate import interp1d

[4.02240000e-03 2.07411200e-01 3.06996600e-01 ... 8.91880195e+04
 8.91881192e+04 8.91883207e+04] [     nan  168.33   168.335 ... 2380.845 2380.85  2380.855]
Оптимальное преобразование: t' = 1.00 * t + 0.00


In [ ]:
dfi1 = None
dfi2 = None
x,y,z = None, None, None
M12 = np.array([[np.cos(dfi1), -(y*np.sin(dfi2)-z*np.cos(dfi2))],[],[]])